# 1. Value Iteration algorithm applied to Taxi in OpenAI GYM

This weeks coding challenge is to use the OpenAI Gym library to build a simple value iteration algorithm for any of the available environments. In this Notebook we're going to implement the algorithm to solve the **Taxi** game from Gym. More information <a href="https://gym.openai.com/envs/Taxi-v1/">here</a>.

## 1.1 How to play the game?

It's a simple game. At least for humans :) 

Let's see it. You have a Taxi. In a little "city". You also have the direction of a passenger and a destination. Of course, you have to go to the position of your passenger, pick them and go to the destination **in the shortest path**

<img src="src/TheTaxiGame.png">

Hope that with the image it's all clear :D

### 1.1.1 Theory of the game

This task was introduced in [Dietterich2000] to illustrate some **issues in hierarchical reinforcement learning**. There are **4 locations** (labeled by different letters) and your job is to pick up the **passenger at one location** and **drop** him off **in another**. You receive **+20 points for a successful dropoff**, and **lose 1 point for every timestep** it takes. There is also a **10 point penalty for illegal pick-up and drop-off actions**.

## 2. The RL problem in theory [3]

So we have:

- An agent: The Taxi
- An environment: The "city"
- A reward: The points
- A set of states: The positions of the taxi and the passenger and destination
- A set of actions: At every time step, the directions the taxi can go

We are going to apply the **value iteration** algorithm so, first of all it's important to know what is a **value function**

#### Value function:

Often denoted V(s), represent **how good** is a state for an agent to be in. The value function depends on the policy by which the agent picks actions to perform. So, if the agent uses a given policy 𝛑 to select actions, the corresponding value function is given by: 

<img src="src/value_function_1.png" width="500px">

Among all possible value-functions, there exist an optimal value function that has higher value than other functions for all states.

<img src="src/optimal_value_function.png" width="500px">

The optimal policy 𝛑* is the policy that corresponds to optimal value function.

<img src="src/optimal_policy.png" width="500px">

#### Q function:

Now we introduce another function which is the **state-action pair Q function**.

<img src="src/q_func_simple.png" width="200px">

Q(s, a) is an indication for how good it is for an agent to pick action a while being in state s.

To find the optimal --> Q\*(s,a)

The Q\*(s, a) is equal to the summation of immediate reward after performing action a while in state s and the discounted expected future reward after transition to a next state s'.

<img src="src/q_optimal.png" width="500px">

This is the **Bellman equation**


## 3. Value Iteration algorithm

Value iteration **computes the optimal state value function** by **iteratively improving the estimate of V(s)**.

The algorithm **initialize V(s)** to arbitrary **random** values. It repeatedly **updates the Q(s, a) and V(s) values until they convergs**. Value iteration is **guranteed to converge to the optimal values**.

<img src="src/value_iteration_pseudo.png" width="500px">

## 4. Explore the game technically

In [107]:
import gym
from gym import wrappers

In [115]:
#Get the game environment
env_name = 'Taxi-v2'
env_wrapped = gym.make(env_name)
env = env_wrapped.unwrapped

[2017-11-19 17:45:16,461] Making new env: Taxi-v2


Let's explore the possible **states** and **actions**

In [133]:
print('Action space A -> len(A): %d' % env.action_space.n)
print('States space S -> len(S): %d' % env.observation_space.n)

Action space A -> len(A): 6
States space S -> len(S): 500


So we have a set of **6 possible actions**<br>
and a set of **500 possible states**

Not bad for start :)

## 5. Reinforcement Learning for Taxi implementation

## 5.1 Value iteration

- The value function will represent a VALUE for each STATE, initialize it to zeros.

- We define a number of max iterations _(max_iterations)_ to converge and a minimum increasing value _(eps)_ to consider not improving.

**In the training loop:**

1. Store the last value function
2. For each STATE _s_ and For each ACTION _a_ 

    2.1 Iterate the NEXT STATES you can go from that determined state-action pair (_s_,_a_)<br>
    2.2 Store the reward from those NEXT STATES according to the formula: **probability * (reward + PreviousValueFunction[NEXT\_STATE])**<br>

3. **Sum** all those rewards from NEXT STATES<br>
4. **Choose the max** reward of _q(s,a)_ pairs and put it on the actual value function for STATE s

5. If the actual value functions hasn't improved more than the improving threshold, convergence is reached.

In [227]:
def value_iteration(env, gamma = 1.0):

    '''
    Value-iteration algorithm
    '''
    #The value function will represent a VALUE for each STATE, initialize it to zeros.
    v = np.zeros(env.observation_space.n)
    max_iterations = 10000
    display_freq = max_iterations // 10
    eps = 1e-20
    lastDif = float('inf')
    
    #Start training loop
    print('Starting training loop...')
    for i in range(max_iterations):
        if(i % display_freq == 0):
            print('25 first elements of actual value function. An array of number of possible states (%d) elements:\n%s'
                  %(env.observation_space.n,v[0:25]))
            
        prev_v = np.copy(v) #Store the last value function
        for s in range(env.observation_space.n):#For each STATE s
            q_sa = []
            for a in range(env.action_space.n): #For each ACTION a
                next_states_rewards = []
                for next_sr in env.P[s][a]: #Iterate the states you can go from determined state-action pair (s,a)
                    p, s_, r, _ = next_sr #(probability, next_state, reward, done) of the states you can go from (s,a)
                    next_states_rewards.append((p*(r + prev_v[s_]))) #Apply the formula to get the rewards
                q_sa.append(np.sum(next_states_rewards)) #Store the sum of rewards for each pair (s,a)
            v[s] = max(q_sa) #Choose the max reward of (s,a) pairs and put it on the actual value function for STATE s
            
        if(np.abs(np.abs(np.sum(prev_v - v)) - lastDif) < eps): #Check convergence
            print('Value-iteration converged at iteration %d' % (i+1))
            break
        lastDif = np.abs(np.sum(prev_v - v))
    return v    

In [228]:
optimal_value_function = value_iteration(env=env,gamma=1.0)

Starting training loop...
25 first elements of actual value function. An array of number of possible states (500) elements:
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
Value-iteration converged at iteration 19


How does the optimal value functions looks like?

In [235]:
print('Value function shape: %s\nFirst 25 values of the value functions:\n%s' 
      % (optimal_value_function.shape,optimal_value_function[0:25]))

Value function shape: (500,)
First 25 values of the value functions:
[ 359.  233.  275.  212.  107.  233.   65.  128.  191.  107.  275.  128.
   65.  107.   65.  212.  380.  254.  296.  233.  338.  212.  254.  191.
  128.]


So we have a real number for each possible state, seems right.

### 5.1 Extract policy

Because we want our agent to **do something** and our agents **behaves according** to the **policy**, the **policy** could be **random actions** or a **value function**...

Now that we have a (optimal) function value we can **extract** the associated (optimal) policy.

**How:**
Just get the argument that maximize (argmax) the value function for each possible state

In [230]:
def extract_policy(v, gamma = 1.0):
    '''
    Extract the policy given a value-function
    '''
    
    policy = np.zeros(env.observation_space.n) #Policy : array of 0s with as many elements as possible states
    for s in range(env.observation_space.n):
        q_sa = np.zeros(env.action_space.n) # q_sa: array of 0s with as many elements as possible actions
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]: #Iterate the states you can go from determined state-action pair
                #next_sr is a tuple of (probability, next_state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    
    return policy

In [231]:
#Extract the policy for the optimal value function that we've just computed with the value iteration algorithm
optimal_policy = extract_policy(v=optimal_value_function, gamma=1.0)

How does the optimal value functions looks like?

In [237]:
print('Value function shape: %s\nFirst 25 values of the value functions:\n%s' 
      % (optimal_value_function.shape,optimal_policy[0:25]))

Value function shape: (500,)
First 25 values of the value functions:
[ 4.  4.  4.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  5.  0.
  0.  0.  3.  3.  3.  3.  0.]


So we have a number in [0,5] for each state. Guess it!

Yes, it's the (optimal) action to take in each state.

### 5.2 Evaluate policy

In fact it's playing the game.

1. Get in a starting state (reset the environment)
2. Take the (optimal) action according to the state --> Ask the policy!
3. Do it
4. If it's ended. Stop.

\*find each element of this numbered list in the code by the index ;)

In [293]:
def run_episode(env, policy, gamma = 1.0, verbose=True,render = False):
    '''
    Evaluates a policy by using it to run an episode and finding it's total reward
    
    returns: 
    total reward: real value of the total reward received by the agent under policy
    '''
    
    state = env.reset() #1. Reset the environment, starting state.
    total_reward = 0
    step_idx = 0
    
    frames = []
    while True:
       
        if render:
            env.render(mode = 'human')
            
        action = int(policy[state]) #2. Ask the policy for the optimal action!
        state, reward, done, info = env.step(action) #3. Do it!
        if verbose:
            print('State: %d - Reward: %d - Done: %d' %(state, reward, done))
            
        total_reward = total_reward + (gamma ** step_idx * reward)
        step_idx += 1
        
        if done: #4. If it's ended stop
            break

    return total_reward

In [294]:
run_episode(env,optimal_policy,verbose=True,render=True)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

State: 201 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
State: 101 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
State: 1 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
State: 17 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
State: 117 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
State: 137 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
| :_: : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
State: 157 - Reward: -1 - Done: 0
+---------+
|R: | : :G|
| : :_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
State: 57 - Reward: -1 

10.0

Do it a lot of times for evaluating

In [295]:
def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    '''
    Evaluates a policy by running it n times
    
    returns:
    average total reward
    '''
    scores = [run_episode(env,policy,gamma,False,False) for _ in range(n)]
    return np.mean(scores)

In [258]:
n = 10
points = evaluate_policy(env,optimal_policy,gamma=1.0,n=n)
print('We have got %.2f average points in %d games' %(points,n))

We have got 10.20 average points in 10 games


### 5.2.1 Formal evaluation

According to the <a href="https://gym.openai.com/envs/Taxi-v1/">official documentation</a>, **Taxi-v1 defines "solving" as getting average reward of 9.7 over 100 consecutive trials.** 

In [298]:
n = 100
points = evaluate_policy(env,optimal_policy,gamma=1.0,n=n)
print('We have got %.2f average points in %d games' %(points,n))

We have got 8.50 average points in 100 games


After all...

<img src="src/playing_with_neural_nets.jpg" width="250px">

Seriously, check it in the render mode, the taxi driver is awesome! Always takes the shortest path! Don't know what else to do :O

## Future work

- Tweak _gamma_ values and see what happens

## References

[1] Taxi - OpenAI Gym - https://gym.openai.com/envs/Taxi-v1/ <br>
[2] An overview of MAXQ hierarchical reinforcement learning - https://link.springer.com/book/10.1007/3-540-44914-0#page=38<br>
[3] Deep Reinforcement Learning Demysitifed (Episode 2) — Policy Iteration, Value Iteration and Q-learning - https://medium.com/@m.alzantot/deep-reinforcement-learning-demysitifed-episode-2-policy-iteration-value-iteration-and-q-978f9e89ddaa